In [1]:
#Extracting text from pdf using the pdfplumber mpackage
import pdfplumber

pdf_path = 'Paula Browne Scholar Report.pdf'
text_data = ""

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text_data += page.extract_text() + '\n'

#save to a text file
with open('data.txt', 'w', encoding='utf-8') as f:
    f.write(text_data)

In [2]:
#Cleaning: remove extra spaces, line breaks and non-text stuff + lowercase.
import re

with open('data.txt', 'r', encoding='utf-8') as f:
    raw_text = f.read()

cleaned_text = re.sub(r'\s+', ' ', raw_text).strip()

with open('cleaned_data.txt', 'w', encoding='utf-8') as f:
    f.write(cleaned_text)


In [10]:
#Tokenize dataset using Hugging Face Tokenizer and data

from transformers import GPT2Tokenizer
from datasets import Dataset

#Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token  # Set padding token to EOS token

with open('cleaned_data.txt', 'r', encoding='utf-8') as f:
    text = f.read()

#Label your text as a dataset for training
dataset = Dataset.from_dict({'text': [text]}) 

#Tokenise
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True) #get tokens from an existing dataset of words from Hugging Face


Map: 100%|██████████| 1/1 [00:00<00:00, 75.45 examples/s]


In [11]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling, GPT2LMHeadModel
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [12]:
model = GPT2LMHeadModel.from_pretrained('gpt2')

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='no',
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs'

)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset,
)

trainer.train()

`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=3, training_loss=3.600430488586426, metrics={'train_runtime': 35.5007, 'train_samples_per_second': 0.085, 'train_steps_per_second': 0.085, 'total_flos': 759379968000.0, 'train_loss': 3.600430488586426, 'epoch': 3.0})